In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv
/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv
/kaggle/input/equity-post-HCT-survival-predictions/train.csv
/kaggle/input/equity-post-HCT-survival-predictions/test.csv


In [2]:
train= pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
test= pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')
data_dict= pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv')


In [3]:
from sklearn.impute import KNNImputer,SimpleImputer 
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler

In [4]:
#train.info()
#train.head(7)

In [5]:
# Step 1: Handle missing values
train.replace({'N/A': np.nan, 'NaN': np.nan}, inplace=True)  # Convert string "N/A" and "NaN" to real NaN
train.dropna(axis=0, how='all', inplace=True)  # Remove rows where all values are NaN
train.dropna(axis=1, how='all', inplace=True)  # Remove columns where all values are NaN

# Step 2: Convert categorical variables to numerical (if necessary)
categorical_cols = train.select_dtypes(include=['object']).columns  # Select categorical columns
train[categorical_cols] = train[categorical_cols].apply(lambda x: pd.factorize(x)[0])  # Encode categories

# Step 3: Convert numeric columns with potential issues
numeric_cols = train.select_dtypes(include=['number']).columns
train[numeric_cols] = train[numeric_cols].apply(pd.to_numeric, errors='coerce')  # Convert to float, force NaNs where errors occur

# Step 4: Fill remaining missing values (if any)
train.fillna(train.median(), inplace=True)  # Replace NaNs with median of each column

# Step 5: Normalize numeric columns (if needed)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train[numeric_cols] = scaler.fit_transform(train[numeric_cols])


In [6]:
# Step 1: Handle missing values
test.replace({'N/A': np.nan, 'NaN': np.nan}, inplace=True)  # Convert string "N/A" and "NaN" to real NaN
test.dropna(axis=0, how='all', inplace=True)  # Remove rows where all values are NaN
test.dropna(axis=1, how='all', inplace=True)  # Remove columns where all values are NaN

# Step 2: Convert categorical variables to numerical (if necessary)
categorical_cols = test.select_dtypes(include=['object']).columns  # Select categorical columns
test[categorical_cols] = test[categorical_cols].apply(lambda x: pd.factorize(x)[0])  # Encode categories

# Step 3: Convert numeric columns with potential issues
numeric_cols = test.select_dtypes(include=['number']).columns
test[numeric_cols] = test[numeric_cols].apply(pd.to_numeric, errors='coerce')  # Convert to float, force NaNs where errors occur

# Step 4: Fill remaining missing values (if any)
test.fillna(test.median(), inplace=True)  # Replace NaNs with median of each column

# Step 5: Normalize numeric columns (if needed)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
test[numeric_cols] = scaler.fit_transform(test[numeric_cols])


Train/Val Split

In [7]:
X = train.drop(columns= ['efs', 'efs_time'])
y = train['efs']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X,y, train_size= 0.75)

In [9]:
train.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,0.000000,0.090909,0.333333,0.000000,0.333333,1.0,1.0,0.000000,0.333333,1.0,...,0.00,0.333333,0.5,1.0,0.333333,1.0,0.333333,1.0,0.0,0.268542
1,0.000035,0.181818,0.333333,0.142857,0.333333,1.0,1.0,0.142857,0.333333,1.0,...,0.25,0.666667,0.5,1.0,0.333333,1.0,0.666667,1.0,1.0,0.027728
2,0.000069,0.090909,0.333333,0.000000,0.333333,1.0,1.0,0.000000,0.333333,1.0,...,0.25,0.666667,0.5,1.0,0.333333,1.0,0.333333,1.0,0.0,0.124356
3,0.000104,0.272727,0.333333,0.142857,0.333333,1.0,1.0,0.000000,0.333333,1.0,...,0.25,0.333333,0.5,1.0,0.333333,1.0,0.333333,1.0,0.0,0.651918
4,0.000139,0.272727,0.333333,0.000000,0.333333,1.0,1.0,0.000000,0.333333,1.0,...,0.25,0.666667,1.0,1.0,0.333333,1.0,0.333333,1.0,0.0,0.101543


In [10]:

# Step 6: Save cleaned dataset
#df.to_csv('cleaned_dataset.csv', index=False)


Pipeline

In [11]:
pipe = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
    ('scaler', StandardScaler()),
    ('classifier', XGBClassifier(use_label_encoder=True, eval_metric='logloss'))
])

Train the Decision Tree Classifier

In [12]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
                ('scaler', StandardScaler()),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enabl...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [13]:
pipe.score(X_val, y_val)

0.6688888888888889

In [14]:
test.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.5,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
X_test =test
preds = pipe.predict(X_test)
final_df = pd.DataFrame({
    'ID': X_test.index,
    'prediction': preds,
})

final_df.ID = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv").ID.values

final_df.to_csv('submission.csv', index = False)

In [16]:
final_df.ID

0    28800
1    28801
2    28802
Name: ID, dtype: int64